In [4]:
# Article Crawler by Section
import requests 
import json
import sqlite3
import pickle 
import urllib
import time
from bs4 import BeautifulSoup as BS
from timeit import default_timer as timer
from multiprocessing import Process
import random

# Constants - Section ID
sectionId_Politics = 100
sectionId_Economy = 101
sectionId_Society = 102
sectionId_IT = 105
sectionId_Life = 103
sectionId_World = 104

section = {
    '정치' : sectionId_Politics,
    '경제' : sectionId_Economy,
    '사회' : sectionId_Society,
    'IT'   : sectionId_IT,
    '생활' : sectionId_Life,
    '세계' : sectionId_World
}

dbName = {
    '정치' : "article_politics",
    '경제' : "article_economy",
    '사회' : "article_society",
    'IT'   : "article_it",
    '생활' : "article_life",
    '세계' : "article_world"
}

In [5]:
# 데이터베이스 초기화

conn = sqlite3.connect("articles.db", check_same_thread=False)
c = conn.cursor()

c.execute('drop table if exists article_all;')
c.execute('drop table if exists article_politics;')
c.execute('drop table if exists article_economy;')
c.execute('drop table if exists article_society;')
c.execute('drop table if exists article_it;')
c.execute('drop table if exists article_life;')
c.execute('drop table if exists article_world;')

table_scheme = '''(
        idx INTEGER PRIMARY KEY AUTOINCREMENT,
        itemid text NOT NULL UNIQUE,
        json text
)
'''

c.execute('''create table if not exists article_all %s;''' % table_scheme);
c.execute('''create table if not exists article_politics %s;''' % table_scheme);
c.execute('''create table if not exists article_economy %s;''' % table_scheme);
c.execute('''create table if not exists article_society %s;''' % table_scheme);
c.execute('''create table if not exists article_it %s;''' % table_scheme);
c.execute('''create table if not exists article_life %s;''' % table_scheme);
c.execute('''create table if not exists article_world %s;''' % table_scheme);

conn.commit()


In [16]:
# db 조작 명령어

def insert_article(tablename, itemid, json):
    # dict 안의 특수기호로 인해 바로 저장 불가능
    pdata = pickle.dumps(json, pickle.HIGHEST_PROTOCOL)
    cnt = 0
    while True:
        try:
            conn.execute('insert into %s (itemid, json) values (?, :data)' % tablename, (itemid, sqlite3.Binary(pdata)))
            conn.commit()
            break
        except sqlite3.OperationalError:
            if cnt > 5: break
            print("rest %s" % itemid)
            time.sleep(0.002 * random.randint(1, 5))
            cnt += 1
            
        except sqlite3.IntegrityError:
            print("skip %s" % itemid)
            break
    return


def select_article(tablename, limit):
    c.execute("select idx, itemid, json from %s order by idx desc limit %s " % (tablename, limit))
    return [(row[0], row[1], pickle.loads(row[2])) for row in c]

'''
#test
insert_article("article_all", '1233', "{'asdf' : 'asdf\"asdfasdf\"'}")
insert_article("article_all", '1234', "{'asdf' : 'asdf\"asdfasdf\"'}")
insert_article("article_all", '1235', "{'asdf' : 'asdf\"asdfasdf\"'}")
insert_article("article_all", '1236', "{'asdf' : 'asdf\"asdfasdf\"'}")
insert_article("article_all", '1237', "{'asdf' : 'asdf\"asdfasdf\"'}")
print(select_article('article_all', 3))
c.execute("DELETE FROM article_all where 1 > 0;")
conn.commit()
'''

'\n#test\ninsert_article("article_all", \'1233\', "{\'asdf\' : \'asdf"asdfasdf"\'}")\ninsert_article("article_all", \'1234\', "{\'asdf\' : \'asdf"asdfasdf"\'}")\ninsert_article("article_all", \'1235\', "{\'asdf\' : \'asdf"asdfasdf"\'}")\ninsert_article("article_all", \'1236\', "{\'asdf\' : \'asdf"asdfasdf"\'}")\ninsert_article("article_all", \'1237\', "{\'asdf\' : \'asdf"asdfasdf"\'}")\nprint(select_article(\'article_all\', 3))\nc.execute("DELETE FROM article_all where 1 > 0;")\nconn.commit()\n'

In [17]:
class NewsArticle:
    ArticleID = 0
    itemId = None
    ArticleTitle = None
    SectionName = None
    ArticleDate = None
    ThumbnailImageURL = None
    Video = False
    Link = None
    Press = None
    Raw_contents = None
    Contents = []

    # 이니셜라이저 : 각 기사별 json 받아서 데이터 타입에 맞게 저장
    def __init__(self, json):
        self.itemId = json["itemId"]
        self.ArticleID = json["articleId"]
        self.ArticleTitle = json["title"]
        self.ArticleDate = json["standardFullDate"] if 'standardFullDate' in json.keys() else None
        self.ThumbnailImageURL = str(json["imageUrl"])
        self.Video = json["videoType"]
        self.Press = json["officeName"]
        self.Link = "http://m.news.naver.com" + json["linkUrl"]
        self.SectionName = (
            json["sectionName"] if json["sectionName"] else None)
        self.get_contents()
        return
    
    
    # json 스트링 리턴
    def __str__(self):
        d = dict()
        d["ArticleID"] = str(self.ArticleID)
        d["ArticleTitle"] = str(self.ArticleTitle)
        d["SectionName"] = str(self.SectionName)
        d["ArticleDate"] = str(self.ArticleDate)
        d["Press"] = str(self.Press)
        d["ThumbnailImageURL"] = str(self.ThumbnailImageURL)
        d["Video"] = str(self.Video)
        d["Link"] = str(self.Link)
        d["Contents"] = (self.Contents)
        return str(d)
    
    
    # 컨텐츠 내용 파싱
    def get_contents(self):
        #print(self.ArticleTitle)
        texts = get_text(self.Link)
        self.Raw_contents = texts
        self.Contents = jsonify_content(texts)
        return
      
    '''
    
    # 파일 이름 저장 : 날짜 및 아티클 ID 기준
    def getFileName(self):
        fileName = "%s%s" % \
            (self.ArticleDate \
             .replace(":", "") \
             .replace("-", "") \
             .replace(" ", ""), 
             self.ArticleID)
        return fileName
        
        
    # 파일로 기록
    def write(self):
        fileName = self.getFileName()
        directory = "articles/%s" % fileName

        try:
            f = open(directory, "r")
        except IOError:
            if self.SectionName:
                print(self.SectionName)
                with open("articles/" + self.SectionName + fileName, "w") as f:
                    f.write(str(self))
            with open(directory, "w") as f:
                f.write(str(self))
            return
        # if reached : file exists
        raise IOError    
    '''


In [18]:
# get_text : URL(str) ->> BS element
def get_text(URL):
    source_code_from_URL = str(urllib.request.urlopen(URL).read(), "utf-8").replace("<br>", "<br/>").replace("<br/><br/>", "<br/>")
    soup = BS(source_code_from_URL, 'html.parser')
    result = soup.find_all('div', id="dic_area")

    # entertain 뉴스를 레퍼런스로 하는 경우 별도의 핸들링 필요
    if not result:
        #print(URL)
        source_code_from_URL = str(urllib.request.urlopen(
            URL.replace("m.news.naver.com", "m.entertain.naver.com")).read(), "utf-8").replace("<br>", "<br/>").replace("<br/><br/>", "<br/>")
        soup = BS(source_code_from_URL, 'html.parser')
        result = soup.find_all('div', id="contentArea")
    
    return result[0]

In [19]:
# BS contents ->>  [indexed dictionary, ...]
def jsonify_content(content):

    # BS.contents는 html 태그 안의 모든 first child를 리스트로 반환
    # 태그가 없어도 무방하므로 본문 내용까지 별도의 child로 리턴됨
    # 기사 앞뒤의 빈 칸(탭, 줄바꿈 기호 등)을 잘라내고 br 태그를 제거함
    content_list = content.contents
    x = list(filter(lambda s : s != "<br/>", map(lambda s: str(s).strip(), content_list)))

    # 각 object에 기사 요소의 배치 순서를 지시
    index = 0

    # json_list에 각 기사 요소를 순서대로 담아 리턴

    json_list = []
    # 텍스트 타입에 따라 분류
    # type : text, video, image, link
    for i in x:

        # 잘못 나온 놈은 걸러야 한다. 인덱스 추가 안하고 다음 요소 탐색
        if i == "":
            continue

        # 여기부턴 어쨌든 요소에 속함.
        # HTML 태그가 하나도 없으면 텍스트로 분류
        if not "<" in i:
            json_list.append({'ArticleIndex': index, "ArticleType": "text", 'content' : i})

        # 태그가 있는 요소들은 다음과 같이 분류함:
        else:
            bs = BS(i, 'html.parser')

            # 비디오 태그 : To Be Implemented
            if "<iframe" in i:
                video_url = bs.find('iframe').get('src')
                json_list.append({'ArticleIndex': index, "ArticleType": "text", 'content': video_url})

            elif "video_area" in i:
                #print(i)
                json_list.append({'ArticleIndex': index, "ArticleType": "text", 
                                  'content': "http://news.video.p.rmcnmv.naver.com/owfs_rmc/read/NEWS_2017_05_26_3/531C05F3C304F34C0F330E6E936E1FC3388_muploader_b_480P_854_1024_128.mp4?_lsu_sa_=632577fcc1be6046fdda357b66c57bba4ecf3f1831089f0633875ec1c74a3f2570240aed61357008709b3a129034fbc5effbc03b5726a5f16d6f884f367d5fa70774723c3e1698cc965eedfef0ec4a5a"})
                #video_url = bs.find('video').get('data-src')
                #json_list.append({'ArticleIndex': index, "ArticleType": "video", 'content': video_url})

            # 이미지 태그 : 대부분 span으로 묶여 있음. 사진과 태그 포함
            # 다만 다른 형식에 span이 사용될 수 있으므로 img를 이용함
            elif "<img" in i:
                #print('image')
                img_url = bs.find('img').get('data-src')
                img_tag = bs.find('em', class_='img_desc')
                # 이미지 태그가 없으면 빈 스트링을 저장함.
                if img_tag: img_tag = img_tag.text
                else: img_tag = ""
                json_list.append({'ArticleIndex': index, "ArticleType": "image", 'content': img_url, 'tag' : img_tag})


            # 링크 태그 : a 안에 들어 있음
            # 네이버 기사는 본문 안에 링크 삽입 잘 안하므로 a 안의 요소만 따도 무방
            elif "<a" in i:
                link_url = bs.find('a').get('href')
                link_text = bs.find('a').text
                json_list.append({'ArticleIndex': index, "ArticleType": "link", 'url': link_url, 'content': link_text})

            # 소제목 태그 : strong, em 등의 텍스트 요소를 포함함
            elif any(["<font" in i, "<strong" in i, "<b" in i]):
                json_list.append({'ArticleIndex': index, "ArticleType": "strapline", 'content': bs.text})

            # 가끔 텍스트에 꺾쇠를 집어넣는 똥같은 언론사들이 있음
            # 문과가 또...
            # 이 경우는 텍스트로 분류
            else:
                json_list.append({'ArticleIndex': index, "ArticleType": "text", 'content': bs.text})

        #print(index)
        index += 1
    return json_list

In [20]:
# request first 12 extlements
# sectionId : int --> [list-of Articles]
def requestBySection(sectionName, limit=10, log=False):
    
    assert type(limit) == int
    assert limit >= 1
    
    sectionId = section[sectionName]

    if log:
        print("section Name = %s" % sectionName)
        print("section ID = %s" % sectionId)
    
    First_URL = 'http://m.news.naver.com/section/moreItemList.json?' + "sectionId=%s" % sectionId
    response = requests.get(First_URL)
    result = parse_response(response, log=log)
    
    articleList = result['articles']
    
    if log:
        for article in result['articles']: 
            print("article ID : %s" % article['itemId'])
        
    for i in range(1, limit):
        data = dict()
        data['sectionId'] = sectionId
        data['componentId'] = result['componentId']
        data['itemId'] = result['articles'][-1]['itemId']
        
        # 파일 이름 비교
        
        URL = "http://m.news.naver.com/section/moreItemList.json"
        response = requests.post(URL, data=data)
        result = parse_response(response, log=log)
        
        if log:
            print("index : %s" % i)
            for article in result['articles']: 
                print("Item ID : %s" % article['itemId'])

        #for article in result['articles']:
        #    print(article['itemId'])
        articleList = articleList + result['articles']
        
    return articleList

In [21]:
# response --> {componentId: int, pageNavigation: int, articles: [article, ...]}
def parse_response(response, log=False):
    
    if response.status_code == 200:
        result = response.json()['message']
        if result["success"]:
            #print(result['contents']['articles'][0].keys())
            result = result['contents']

            if log:
                print()
                print("-- Article log start --")
                print(result['componentId'])
                print(result['pageNavigation'])
                for article in result['articles']:
                    for (k, v) in article.items():
                        print ("- %s : %s" % (k, v))
                    print()
                print("-- Article log end --")
                print()
                
            return result

        else:
            print("Error with Naver request %s" % result["success"])
            return False
    else:
        print("Error with HTTP code %s" % response.status_code)
        return False

In [22]:
# 메인화면 뉴스묶음 크롤링은 별도로 구현
def request_list(k):
    data = {"page": str(k)}
    navernews_url = "http://m.news.naver.com/mainNews/moreMainNews.json"
    r = requests.post(navernews_url, data=data)
    data = json.loads(r.text)
    #articleList = []
    #for articleData in data['message']['itemList']:
    #    # data의 데이터 선택 및 article 클래스 생성
    #    article = NewsArticle(articleData)
    #    articleList.append(article)
    #return articleList
    return data["message"]['itemList']

def requestMainSection(k):
    l = list()
    for i in range(k):
        l = l + request_list(i)
    return l



In [23]:
# 데이터베이스 삽입
def section_newsCrawler(sectionName, limit):
    sectionId = section[sectionName]
    tableName = dbName[sectionName]
    l = requestBySection(sectionName, limit=limit, log=False)
    # 시간 역순
    l.reverse()
    for i in l:
        j = NewsArticle(i)
        insert_article(tableName, j.itemId, str(j))
    return

def main_newsCrawler(limit):
    l = requestMainSection(limit)
    l.reverse()
    for i in l:
        j = NewsArticle(i)
        insert_article('article_all', j.itemId, str(j))
    return

In [24]:
def crawl(limit):
    processes = []
    
    p = Process(target=main_newsCrawler, args=(limit, ))
    p.start()
    processes.append(p)

    for sectionName in section.keys():
        p = Process(target=section_newsCrawler, args=(sectionName, limit))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

In [ ]:
start = timer()
crawl(40)
end = timer()
print(end - start)

while True:
    start = timer()
    crawl(40)
    end = timer()
    print(end - start)
    time.sleep(100)

skip 2662940
skip 2662944
skip 2662941
skip 2662942
skip 2662945
skip 2663242
skip 2662970
skip 2662969
skip 2662968
skip 2662955
skip 2662956
skip 2662957
skip 2662958
skip 2662959
skip 2662960
skip 2662961
skip 2662962
skip 2662963
skip 2662964
skip 2662965
skip 2662966
skip 2662953
skip 2662952
skip 2663003
skip 2663008
skip 2663445
skip 2663007
skip 2663006
skip 2662971
skip 2663331
skip 2662967
skip 2662950
skip 2662954
skip 2663280
skip 2662990
skip 2662972
skip 2663594
skip 2663031
skip 2662997
skip 2662975
skip 2663448
skip 2663215
skip 2663114
skip 2663382
skip 2663388
skip 2663092
skip 2663024
skip 2663467
skip 2663041
rest 2344378
skip 2663040
skip 2663050
rest 2120368
rest 49395201
rest 34790973
rest 26044411
skip 2663392
rest 49510336
skip 2663015
skip 2663365
skip 2663379
skip 2663085
skip 2663093
skip 2663094
skip 2663361
skip 2663483
skip 2663028
skip 2663265
skip 2663305
skip 2662984
skip 2663446
skip 2663113
skip 2663029
skip 2663039
skip 2344378
skip 2344450
skip 234

skip 34796899
skip 34796897
skip 34796898
skip 34796896
rest 2662986
skip 34795913
skip 34796895
skip 34796894
skip 34796893
rest 2349025
skip 34796845
skip 34796352
skip 34796869
skip 34796926
skip 34796902
skip 34796901
skip 34798028
skip 34796934
skip 34796933
skip 34796932
skip 34796931
skip 34796930
skip 34796929
skip 34796928
skip 34796927
skip 34797475
skip 34797474
skip 34797569
skip 34797649
skip 34797650
skip 34797647
skip 34797646
skip 34798219
skip 34798218
skip 34797932
skip 34798072
skip 34798027
skip 34798216
skip 34798242
skip 34798460
skip 34798459
skip 34799086
skip 34798484
skip 34798672
skip 34803228
skip 34798874
skip 34798873
skip 34799040
rest 2662986
skip 34799038
skip 34799037
skip 34799036
skip 34799033
rest 2349025
skip 34799034
skip 34799501
skip 34799131
skip 34799176
skip 34799288
skip 34811229
skip 34799311
skip 34799382
skip 34799359
skip 34799502
skip 34799429
skip 34799383
skip 34799477
skip 34799551
skip 34799599
skip 34799526
skip 34799575
skip 34799

skip 2121541
skip 2121534
skip 2121868
skip 2121866
skip 2121765
skip 2121764
skip 2122279
skip 2122278
skip 2122204
skip 2122277
skip 2122203
skip 2122276
rest 2663484
skip 2122197
skip 2122270
skip 2122269
skip 2122268
skip 2122267
skip 2122308
skip 2122241
skip 2122262
skip 2122307
skip 2122541
skip 2122301
skip 2122328
skip 2122326
skip 2122323
skip 2122322
skip 2122375
skip 2122373
skip 2122372
skip 2122557
skip 2122392
skip 2122385
skip 2122384
skip 2122391
skip 2122505
rest 49395203
rest 26044422
skip 2122566
rest 49510347
skip 2122565
skip 2122564
skip 2122563
skip 2122631
skip 2122567
skip 2122678
skip 2122630
skip 2122696
skip 2123079
skip 2123262
skip 2123359
skip 2122894
skip 2122893
skip 2122892
skip 2122796
skip 2122886
skip 2122885
skip 2122884
rest 2663484
skip 2122883
skip 2122882
skip 2122714
skip 2123035
rest 2349018
skip 2122980
skip 2122992
skip 2124037
skip 2123072
skip 2123071
skip 2123070
skip 2123149
skip 2123148
skip 2123147
skip 2123146
skip 2123139
skip 2123

rest 26044421
rest 49395202
skip 49510607
skip 49510606
skip 49510605
skip 49510604
skip 49510673
skip 49510603
skip 49510602
skip 49510601
skip 49510600
skip 49510599
skip 49510598
skip 49510609
skip 49510594
skip 49510595
skip 49510593
skip 49510592
skip 49510618
skip 49510620
skip 49510623
rest 2663440
skip 49510629
skip 49510628
rest 2349287
skip 49510627
skip 49510626
skip 49510625
skip 49510624
skip 49510631
skip 49510630
skip 49510633
skip 49510632
skip 49510637
skip 49510654
skip 49510645
skip 49510644
skip 49510642
skip 49510643
skip 49510646
skip 49510641
skip 49510650
skip 49510649
skip 49510651
skip 49510656
skip 49510653
skip 49510657
rest 26044421
rest 49395202
skip 49510655
skip 49510706
skip 49510658
skip 49510661
skip 49510938
skip 49510660
skip 49510659
skip 49510663
skip 49510662
skip 49510664
skip 49510669
skip 49510670
skip 49510675
skip 49510676
skip 49510674
skip 49510680
rest 2663440
skip 49510865
skip 49510702
skip 49510679
skip 49510681
skip 49510678
skip 4951

skip 2349281
skip 2349294
skip 2349280
skip 2349189
skip 2349193
skip 2349192
skip 2349231
skip 2349330
skip 2349329
skip 2349328
skip 2349277
skip 2349327
skip 2349324
skip 2349288
skip 2349325
skip 2349326
skip 26044420
skip 26044451
skip 26044442
skip 26044424
skip 26044447
skip 26044471
skip 26044472
skip 26044490
skip 26044515
skip 26044509
skip 26044532
skip 26044503
skip 26044517
skip 26044530
skip 26044528
skip 26044538
skip 26044573
skip 26044562
skip 26044564
skip 26044565
skip 26044539
rest 2663611
skip 26044563
skip 26044583
skip 26044574
skip 26044614
skip 26044589
skip 26044613
skip 26044592
skip 26044590
skip 26044625
skip 26044624
skip 26044622
skip 26044623
skip 26044646
skip 26044645
skip 26044653
skip 26044668
skip 26044651
skip 26044660
skip 26044680
skip 26044693
skip 26044687
skip 26044734
skip 26044737
skip 26044710
skip 26044739
rest 49395543
skip 26044746
skip 26044727
skip 26044753
skip 26044769
skip 26044752
skip 26044770
skip 26044797
skip 26044786
skip 2604

...

skip 49510812
skip 49510818
skip 49510839
skip 49510814
skip 49510807
skip 49510817
skip 49510816
skip 49510815
skip 49510825
skip 49510822
skip 49510829
skip 49510828
skip 49510827
rest 2120077
rest 49395209
rest 34790944
skip 49510831
rest 2344417
rest 26044424
skip 49510834
skip 49510838
skip 49510841
skip 49510837
skip 49510836
skip 49510835
rest 2663633
skip 49510845
skip 49510842
skip 49510843
skip 49510848
skip 49510857
skip 49510846
skip 49510856
skip 49510847
skip 49510854
skip 49510859
skip 49510855
skip 49510858
skip 49510861
skip 49510860
skip 49510862
skip 49510863
skip 49510878
skip 49510877
skip 49510866
skip 49510875
skip 49510874
skip 49510872
skip 49510880
skip 49510871
skip 49510879
skip 49510869
skip 49510870
skip 49510867
skip 49510868
skip 49510884
skip 49510887
skip 49510888
skip 49510883
rest 2120077
skip 49510908
rest 49395209
rest 34790944
rest 2344417
rest 26044424
skip 49510886
skip 49510892
skip 49511029
skip 49510902
skip 49510903
skip 49510904
skip 495109

skip 26047392
skip 26047391
skip 26047426
skip 26047485
skip 26047558
skip 26047557
skip 26047637
skip 26047638
skip 26047639
skip 26047645
skip 26047671
skip 26047672
skip 26047803
skip 26047673
skip 26047789
skip 26047674
skip 26047677
skip 26047690
skip 26047684
skip 26047683
skip 26047682
rest 2120080
skip 26047681
rest 34791085
rest 49395208
rest 2344449
skip 26047680
skip 26047679
skip 26047678
skip 26047754
skip 26047879
skip 26047788
skip 26047872
skip 26047776
skip 26047777
skip 26047775
skip 26047751
skip 26047752
skip 26047755
skip 26047768
skip 26047756
skip 26047757
skip 26047769
skip 26047795
skip 26047869
skip 26047868
skip 26047856
skip 26047857
skip 26047855
skip 26047865
skip 26047864
skip 26047863
skip 26047871
skip 26047870
skip 26047878
skip 26047881
skip 26047910
skip 26047990
skip 26047903
skip 26047902
skip 26047901
skip 26047900
skip 26047898
skip 26047899
skip 26047909
rest 2120080
skip 26047916
rest 49395208
rest 34791085
rest 2344449
skip 26047929
skip 26047

skip 2349134
skip 2349135
rest 2663638
skip 2349117
skip 2349139
rest 26048128
skip 2349116
skip 2349281
skip 2349294
skip 2349280
skip 2349189
skip 2349193
skip 2349192
skip 2349231
skip 2349330
skip 2349329
skip 2349328
skip 26048128
skip 26048164
skip 26048163
skip 26048283
skip 26048245
skip 26048244
skip 26048268
skip 26048256
skip 26048267
skip 26048282
skip 26048294
skip 2663638
skip 2663647
skip 2663646
skip 2663640
skip 2663719
skip 2663686
skip 2663675
skip 2663689
skip 2663651
rest 2120197
rest 49395220
skip 2663655
rest 34792575
skip 2663659
skip 2663656
skip 2663707
skip 2349277
skip 2349327
skip 2349324
skip 2349288
skip 2349325
skip 2349326
skip 2663687
skip 2663688
skip 2663661
skip 2663665
skip 2663666
skip 2663694
skip 2663681
skip 2663669
skip 2663667
skip 2663676
skip 2663693
skip 2663680
skip 2663703
skip 2663725
skip 2663724
skip 49395220
skip 49395214
skip 49395216
skip 49395215
skip 49395224
skip 49395222
skip 49395226
skip 49395269
skip 49395227
skip 49395229
s

skip 2120512
skip 2120643
skip 2120644
skip 2120546
skip 2120555
skip 2120549
skip 2120630
skip 2120662
skip 2120664
skip 2120663
skip 2120719
skip 2120746
skip 2120747
skip 2120793
skip 2120818
skip 2120817
skip 2120792
skip 2120791
skip 2120806
skip 2120805
skip 2120804
skip 2120890
skip 2120884
rest 2663715
skip 2120918
skip 2120997
skip 2120994
skip 2120964
skip 2121410
skip 2120957
rest 34791236
skip 2121042
skip 2121041
skip 2121030
skip 2121029
skip 2121178
skip 2121118
skip 2121176
skip 2121175
skip 2121153
skip 2121257
skip 2121387
skip 2121304
skip 2121325
skip 2121403
skip 2121402
skip 2121401
skip 2121392
skip 2121400
skip 2121393
skip 2121391
skip 2121390
skip 2121389
skip 2121388
skip 2121462
skip 2121413
skip 2121412
skip 2121411
skip 2121409
skip 2121470
skip 2121472
skip 2121469
skip 2121468
skip 2121541
skip 2121534
rest 2663715
skip 2121868
skip 2121866
skip 2121765
skip 2121764
skip 2122279
skip 2122278
rest 34791236
skip 2122204
skip 2122277
skip 2122203
skip 21222

skip 34802978
skip 34802977
skip 34803413
skip 34802952
skip 34803004
skip 34803003
skip 34803202
skip 34803175
skip 34803054
skip 34803608
skip 34803232
skip 34803282
skip 34803410
skip 34803412
skip 34803409
skip 34805483
skip 34803546
rest 2663729
skip 34803466
skip 34803573
skip 34803576
skip 34803574
skip 34803606
skip 34803607
skip 34803686
skip 34803685
skip 34803952
skip 34803764
skip 34805257
skip 34803763
skip 34803815
skip 34803894
skip 34804089
skip 34804189
skip 34804088
skip 34804141
skip 34804140
skip 34804188
skip 34804139
skip 34804546
skip 34804445
skip 34804444
skip 34804443
skip 34804647
skip 34805147
skip 34805232
skip 34805306
rest 2663729
skip 34805456
skip 34805457
skip 34805510
skip 34805561
skip 34805484
skip 34808721
skip 34806240
skip 34806410
skip 34806093
skip 34806239
skip 34807288
skip 34807287
skip 34807146
skip 34808029
skip 34808694
skip 34808720
skip 34808917
skip 34808844
skip 34808843
skip 34811131
skip 34810791
skip 34809616
skip 34810840
skip 348